In [ ]:
import os
import pandas as pd
import pickle
import numpy as np
from sklearn.preprocessing import StandardScaler

folder = "/Users/elizabeth 1/Library/CloudStorage/Box-Box/Wu Lab/Project - 2024/IN Carta [EB]/Results CSVs/11Plates_EDA"
# Create & Save combined dataframes
# Open the file and load the dictionary
with open("Results CSVs/11Plates_EDA/normalized_dict.pkl", 'rb') as file:
    normalized_dict = pickle.load(file)

combined_dict = {}
csv_dict = {}

# Find new subfolders to process
new_subfolders = []
for dir_name in next(os.walk(folder))[1]:
    combined_green_path = os.path.join(folder, dir_name, 'green_combined.csv')
    combined_red_path = os.path.join(folder, dir_name, 'red_combined.csv')
    if dir_name not in normalized_dict or not (
            os.path.exists(combined_green_path) and os.path.exists(combined_red_path)):
        new_subfolders.append(dir_name)

# Process new subfolders
for dir_name in new_subfolders:
    csv_dict[dir_name] = {}
    folder_path = os.path.join(folder, dir_name)
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if "Red" in file_name:
            name = "red"
        elif "Green" in file_name:
            name = "green"
        if "Cell" in file_name:
            name += "_cells"
        elif "Nuclei" in file_name:
            name += "_nuclei"
        csv_dict[dir_name][name] = pd.read_csv(file_path)

# Drop unnecessary columns and combine nuclei & cell data
columns_to_drop = [0, 4, 5]
for plate in new_subfolders:
    for result, df in csv_dict[plate].items():
        df.drop(df.columns[columns_to_drop], axis=1, inplace=True)
        if "nuclei" in result:
            df.drop(df.columns[[0, 1, 2]], axis=1, inplace=True)
    combined_dict[plate] = {
        'green': pd.concat([csv_dict[plate]['green_cells'], csv_dict[plate]['green_nuclei']], axis=1),
        'red': pd.concat([csv_dict[plate]['red_cells'], csv_dict[plate]['red_nuclei']], axis=1)
    }

# Save combined data to CSV only for new subfolders
for plate in new_subfolders:
    for color, df in combined_dict[plate].items():
        output_path = os.path.join(folder, plate, f"{color}_combined.csv")
        df.to_csv(output_path, index=False)


# remove non-numerical data & fix well name
def modify_string(s):
    return s[:1] + s[4:]


for plate in combined_dict:
    for result in combined_dict[plate]:
        df = combined_dict[plate][result]
        df.drop(df.columns[[1, 2]], axis=1, inplace=True)
        df['WELL LABEL'] = df['WELL LABEL'].apply(modify_string)
        df.set_index('WELL LABEL', inplace=True)

# Save to a pickle file
with open(
        '/Users/elizabeth 1/Library/CloudStorage/Box-Box/Wu Lab/Project - 2024/IN Carta [EB]/Results CSVs/11Plates_EDA/nuclei_and_cell_combined_dict.pkl',
        'wb') as f:
    pickle.dump(combined_dict, f)
# Normalization
scaler_standard = StandardScaler()

for plate in new_subfolders:
    for color, df in combined_dict[plate].items():
        normalized_dict[plate][color] = pd.DataFrame(scaler_standard.fit_transform(df), columns=df.columns)
        output_path = os.path.join(folder, plate, f"{color}_normalized.csv")
        df.to_csv(output_path)

with open(
        '/Users/elizabeth 1/Library/CloudStorage/Box-Box/Wu Lab/Project - 2024/IN Carta [EB]/Results CSVs/11Plates_EDA/normalized_dict.pkl',
        'wb') as f:
    pickle.dump(normalized_dict, f)